In [61]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
if all(item in df.columns for item in ['curation_status', 'curation_message']):
  df.fillna(value={'curation_status': 0, 'curation_message': ''}, inplace=True)
# else:
# df['curation_status'] = 0
# df['curation_message'] = ''
df.tail(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
498,Buatlah website blog tentang wisata di Indones...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog ""Kesehatan dan Kecantik...",0.303761,0.0,
499,"Buatlah website informasi ""Wisata Kuliner"" den...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website travel agency dengan tema ""E...",0.301174,0.0,
500,"Buatlah website komunitas ""Hobi Saya"" dengan d...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website informasi ""Wisata Kuliner"" d...",0.281707,0.0,


In [62]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [63]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break
clear_output()

In [64]:
curation_df

,curation_status,curation_message
498,1,
499,1,
500,1,


In [65]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
498,Buatlah website blog tentang wisata di Indones...,"<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website blog ""Kesehatan dan Kecantik...",0.303761,1,
499,"Buatlah website informasi ""Wisata Kuliner"" den...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website travel agency dengan tema ""E...",0.301174,1,
500,"Buatlah website komunitas ""Hobi Saya"" dengan d...","<!DOCTYPE html>\n<html lang=""id"">\n\n<head>\n ...","{'Buatlah website informasi ""Wisata Kuliner"" d...",0.281707,1,


In [66]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")
print('regen-curated.json exported sucessfully!')

regen-curated.json exported sucessfully!


In [3]:
ok_df = pd.read_json('regen.json')
ok_df[['instruction', 'output']].to_json('webgen-dataset-2.json', orient='records')
print('webgen-dataset-2.json exported sucessfully!')

webgen-dataset-2.json exported sucessfully!
